<hr style="height: 1px;">
<i>This notebook was authored by the 8.S50x Course Team, Copyright 2022 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

<h1>Lesson 6: Confidence</h1>


<a name='section_6_0'></a>
<hr style="height: 1px;">


## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.0 Overview</h2>


<h3>Navigation</h3>

<table style="width:100%">
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_1">L6.1 Introduction to Confidence Intervals</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_1">L6.1 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_2">L6.2 z-Scores and Confidence Intervals for Other Distributions</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_2">L6.2 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_3">L6.3 Another Example and Rules of Significance</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_3">L6.3 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_4">L6.4 Moments of Distributions and Mapping</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_4">L6.4 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_5">L6.5 Monte Carlo Integration</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_5">L6.5 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_6">L6.6 Returning to Fitting Supernova Data</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_6">L6.6 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_7">L6.7 Fitting with a More Accurate Model</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_7">L6.7 Exercises</a></td>
    </tr>
    <tr>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#section_6_8">L6.8 Fit to Full Cosmological Model</a></td>
        <td style="text-align: left; vertical-align: top; font-size: 10pt;"><a href="#exercises_6_8">L6.8 Exercises</a></td>
    </tr>
</table>



<h3>Learning Objectives</h3>

In this Lesson, we will spend some time further understanding uncertainty in more complicated scenarios, exploring the following objectives:


- How do we quote significance?
- Asymmetric distributions
- Moments of distributions
- Numerical Integration
- A more sophisticated fit
- What have we learned about the properties of the Universe

<h3>Importing Data (Colab Only)</h3>

If you are in a Google Colab environment, run the cell below to import the data for this notebook. Otherwise, if you have downloaded the course repository, you do not have to run the cell below.

See the source and attribution information below:

>data: data/L04/sn_z_mu_dmu_plow_union2.1.txt <br>
>source: http://supernova.lbl.gov/Union/, https://arxiv.org/abs/1105.3470 <br>
>attribution: The Supernova Cosmology Project, arXiv:1105.3470v1 <br>
>license type: https://arxiv.org/licenses/nonexclusive-distrib/1.0/license.html <br>

In [ ]:
#>>>RUN: L6.0-runcell00

#importing data from git repository

!git init
!git remote add -f origin https://github.com/mitx-8s50/nb_LEARNER/
!git config core.sparseCheckout true
!echo 'data/L04' >> .git/info/sparse-checkout
!git pull origin main

<h3>Importing Libraries</h3>

Before beginning, run the cells below to import the relevant libraries for this notebook.


In [ ]:
#>>>RUN: L6.0-runcell01

!pip install lmfit

In [ ]:
#>>>RUN: L6.0-runcell02

import numpy as np                #https://numpy.org/doc/stable/
from scipy import stats           #https://docs.scipy.org/doc/scipy/reference/stats.html
from scipy import optimize as opt #https://docs.scipy.org/doc/scipy/reference/optimize.html
import matplotlib.pyplot as plt   #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
import math                       #https://docs.python.org/3/library/math.html
import csv                        #https://docs.python.org/3/library/csv.html
import lmfit                      #https://lmfit.github.io/lmfit-py/ 

<h3>Setting Default Figure Parameters</h3>

The following code cell sets default values for figure parameters.

In [ ]:
#>>>RUN: L6.0-runcell03

#set plot resolution
%config InlineBackend.figure_format = 'retina'

#set default figure parameters
plt.rcParams['figure.figsize'] = (9,6)

medium_size = 12
large_size = 15

plt.rc('font', size=medium_size)          # default text sizes
plt.rc('xtick', labelsize=medium_size)    # xtick labels
plt.rc('ytick', labelsize=medium_size)    # ytick labels
plt.rc('legend', fontsize=medium_size)    # legend
plt.rc('axes', titlesize=large_size)      # axes title
plt.rc('axes', labelsize=large_size)      # x and y labels
plt.rc('figure', titlesize=large_size)    # figure title


<a name='section_6_1'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.1 Introduction to Confidence Intervals</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_0) | [Exercises](#exercises_6_1) | [Next Section](#section_6_2) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2024/block-v1:MITxT+8.S50.1x+3T2024+type@sequential+block@seq_LS6/block-v1:MITxT+8.S50.1x+3T2024+type@vertical+block@vert_LS6_vid1" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Slides</h3>

Run the code below to view the slides for this section, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L06/slides_L06_01.html" target="_blank">HERE</a>.

In [ ]:
#>>>RUN: L6.1-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L06/slides_L06_01.html', width=970, height=550)

<h3>Overview</h3>

For the past few Lessons, we have been considering uncertainties that we have defined to be the variance of a distribution, but what exactly do they mean? Let's go back to the p-value definition, given a pdf $p\left(x|\theta\right)$

$$
\begin{eqnarray}
P(x|\theta,x\in \Delta) & = &  \int_{x}^{x+\Delta} p\left(x|\theta\right) dx \\
P_{right}(x|\theta, x\leq x_{0}) & = &  \int_{-\infty}^{x_{0}} p\left(x|\theta\right) dx
\end{eqnarray}
$$

The bottom integral is the cumulative distribution function (cdf), we can use this to derive relationships to the variances of distributions. Simply put, we can compute the probability within various intervals of a distribution.  For the normal (Gaussian) distribution variable $x$, we typically transform $x$ so that it can be written in terms of a normal distribution of unit $1$.


$$
\begin{equation}
\mathcal{N}(0,1) = \frac{x-\bar{x}}{\sigma}
\end{equation}
$$
The above shifts $x$ to be center about 0, and rescales the data so that the standard deviation is 1. 

Let's play around with a few things. First, let's compute the probability that an event falls 2 standard deviations outside of a Gaussian distribution. We can do this by using the CDF for -2 and 2 standard deviations, and noting that the integral of a probability distribution is 1.  

In [ ]:
#>>>RUN: L6.1-runcell01

#probability for events within 1 sigma
pM1=stats.norm.cdf(-1)
p1=stats.norm.cdf(1)
print("probability for events within 1 sigma: ", p1-pM1)
print()

#probability for events strictly < 1 sigma upper bound
pM1=stats.norm.cdf(-1)
p1=stats.norm.cdf(1)
print("probability for events strictly < 1 sigma upper bound: ", p1)
print()

#probability for events within 2 sigma
pM2=stats.norm.cdf(-2)
p2=stats.norm.cdf(2)
print("probability for events within 2 sigma: ", p2-pM2)
print()

#probability for events strictly > 2 sigma upper bound
pM2=stats.norm.cdf(-2)
p2=stats.norm.cdf(2)
print("probability for events strictly > 2 sigma upper bound: ", 1-p2)
print()

<a name='exercises_6_1'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_1) | [Next Section](#section_6_2) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.1.1</span>

Using stats.norm.cdf(), calculate the p value corresponding to $x$ being within $\pm 3\sigma$ of the mean in a normal distribution. Enter your answer as a number (not a percentage) with precision 1e-4.


In [ ]:
#>>>EXERCISE: L6.1.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#YOUR CODE HERE


>#### Follow-up 6.1.1a (ungraded)
>
>What is the delta log likelihood that corresponds to a 3 sigma deviation?

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.1.2</span>

When computing p-values, we often refer to the region of the distribution that we are working with as "left-handed" or "right-handed". For instance, if we wanted to know the probability of a left-handed $3\sigma$ deviation, this would be the area under the curve (the CDF) to the left of the lower $3\sigma$ bound.

What is the probability for a right-handed 5$\sigma$ deviation? What is the probability of ANY deviation greater than 5$\sigma$?

Enter your answer as a list of two numbers (not a percentage) times 1e-7, with precision 1e-3. For instance, an answer of `6.238572e-7` would be reported as `6.238`.
    
Your list should be `[p_right_handed_5sigma, p_any_5sigma]`.

In [ ]:
#>>>EXERCISE: L6.1.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#YOUR CODE HERE


<a name='section_6_2'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.2 z-Scores and Confidence Intervals for Other Distributions</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_1) | [Exercises](#exercises_6_2) | [Next Section](#section_6_3) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2024/block-v1:MITxT+8.S50.1x+3T2024+type@sequential+block@seq_LS6/block-v1:MITxT+8.S50.1x+3T2024+type@vertical+block@vert_LS6_vid2" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Overview</h3>

We often want to characterize our probability by something that is easier to speak about in terms of deviations. Because of the central limit theorem and everything essentially becoming a Gaussian distribution in the large $N$ limit, we tend to refer to probabilities in units of standard deviations ($\sigma$) of a normal distribution. A 1$\sigma$ deviation thus refers to a deviation that is within the x$\pm1\sigma$ of the mean for a Gaussian distribution, or in other words with 1-68.2% = 31.8% probability that a fluctuation is larger than the observed fluctuation. 

To compute these probabilities, we can gain rely on the CDF distribution of the Gaussian distribution. Let's compute the probabilities for 1 to 5 standard deviations.  


In [ ]:
#>>>RUN: L6.2-runcell01

from scipy import stats

#Let's do some integrals
p50=stats.norm.cdf(0)
p1=stats.norm.cdf(1)
p2=stats.norm.cdf(2)
p3=stats.norm.cdf(3)
p5=stats.norm.cdf(5)
pM1=stats.norm.cdf(-1)
pM2=stats.norm.cdf(-2)
pM3=stats.norm.cdf(-3)
pM5=stats.norm.cdf(-5)
#print(p50,p1,p2,p3,pM1,pM2,pM3)

#Whats the probability of things fluctuation more that 1\sigma
print(p1-pM1,"within 1 standard deviations")
print(p2-pM2,"within 2 standard deviations")
print(p3-pM3,"within 3 standard deviations")
print(p5-pM5,"within 5 standard deviations")

#Sometimes we only consider 1-sided p-values
print((1.-p1),"to fluctuate above 1 standard deviation")
print((1.-p3),"to fluctuate above 3 standard deviation")
print((1.-p5),"to fluctuate above 5 standard deviation")

These probability values define what we call confidence intervals. We also often write these as z-scores. For a measurement, the z-score is the probability that a measurement is within $z$ standard deviations of a distribution. 

$$
\begin{equation}
\bar{x}\pm z \sigma
\end{equation}
$$

Z-scores are often considered in the context of Gaussian distributions with z-scores corresponding to 68%, 95%, and 99.75% for 1,2, and 3$\sigma$ deviations, respectively. However, for more complicated distributions, the approach is a bit different. 

<h3>Asymmetric distributions</h3>

In the discussion above, we considered the simplest scenario where we looked at the variation of distributions which we assumed were symmetrically distributed about zero. What if these distributions are asymmetric, how do we define the variations? Let's take a look at some asymmetric distributions. 

In the code below, we are going to use a few additional functions:
`pdf` $\rightarrow$ this computes the pdf value, `ppf` $\rightarrow$ this goes the reverse direction of the cdf function by giving the input value (i.e. $\sigma$ value) for a specific probability. Let's do this for a $\chi^{2}$ distribution.

In [ ]:
#>>>RUN: L6.2-runcell03

import numpy as np
import matplotlib.pyplot as plt
#code from here(© Eric Kim): https://aegis4048.github.io/comprehensive_confidence_intervals_for_python_developers

#Let's plot a chi2 with 9 degrees of freedom (df)
df = 9 
x = np.linspace(-1, 28, 1000)
y = stats.chi2.pdf(x, df, loc=0, scale=1)

# two-tailed
#Note we will use this function percent point function(ppf), 
#which inverts the cdf and gives a z from a probability
two_right_tail = stats.chi2.ppf(1 - 0.025, df) #left value
two_left_tail  = stats.chi2.ppf(1 - 0.975, df) #right value
print("two tail values:",two_right_tail,two_left_tail)

# one tailed
one_right_tail = stats.chi2.ppf(1 - 0.05, df)
one_left_tail  = stats.chi2.ppf(1 - 0.95, df)
print("one tail values:",one_right_tail,one_left_tail)

plt.style.use('seaborn-whitegrid')
fig, axes = plt.subplots(1, 3, figsize=(12, 3))

for ax in axes:
    ax.plot(x, y, c='black')
    ax.grid(False)
    #ax.xaxis.set_major_formatter(plt.NullFormatter())
    #ax.yaxis.set_major_formatter(plt.NullFormatter())

#now let's fill this from the left
axes[0].fill_between(x, 0, y, where=(np.array(x) > min(x)) & (np.array(x) <= two_left_tail), facecolor='grey')
axes[0].fill_between(x, 0, y, where=(np.array(x) > two_left_tail) & (np.array(x) < two_right_tail), facecolor='lightgrey')
axes[0].fill_between(x, 0, y, where=(np.array(x) > two_right_tail) & (np.array(x) <= max(x)), facecolor='grey')

axes[1].fill_between(x, 0, y, where=(np.array(x) > min(x)) & (np.array(x) < one_right_tail), facecolor='lightgrey')
axes[1].fill_between(x, 0, y, where=(np.array(x) > one_right_tail) & (np.array(x) <= max(x)), facecolor='grey')

axes[2].fill_between(x, 0, y, where=(np.array(x) > min(x)) & (np.array(x) <= one_left_tail), facecolor='grey')
axes[2].fill_between(x, 0, y, where=(np.array(x) > one_left_tail) & (np.array(x) <= max(x)), facecolor='lightgrey')

fig.tight_layout()

Now that we have taken a look at these distributions, we see a clear asymmetry in the lengths of the z-values to p-values of the left and right. What is conserved is the integrals, but the values are different. Let's go ahead and compute the mean and variance of these distributions. To do this, we will sample a distribution using some new functions, denoted `rvs` ("random variate samples"); this function allows us to sample specific distributions so that we are able to generate toy events off of these sampled events. 

In [ ]:
#>>>RUN: L6.2-runcell04

#Let's compute the mean and RMS of a sample from this distribution
df=9
#Let's sample this distribution
y_chi2 = stats.chi2.rvs(size=1000,df=df)
print(y_chi2[0:5]) #print some valued
print("Sampled Mean:",y_chi2.mean(),"Sampled Stddev:",y_chi2.std())

z=1.5 #Let's deviation corresponding to 1.5sigma in a Gauassian
x = np.linspace(-1, 28, 1000)
y = stats.chi2.pdf(x, df, loc=0, scale=1)
two_right_tail = stats.chi2.ppf(1 - stats.norm.cdf(-z), df)
two_left_tail = stats.chi2.ppf(1 - stats.norm.cdf(z), df)

def plotItAll(x,y,y_chi2,z):
    #Now let's plot the filled area using the true pdfs and the assumed variations if it were a Gaussian
    plt.style.use('seaborn-whitegrid')
    #plot distribution
    plt.plot(x, y, c='black',label="$\chi^2$")
    #plot chi2
    plt.hist(y_chi2, histtype='stepfilled', edgecolor='k', alpha=0.4, color='gray', density=True,bins=20,label="events")
    #true values
    plt.fill_between(x, 0, y, where=(np.array(x) > min(x)) & (np.array(x) <= two_left_tail), facecolor='grey')
    plt.fill_between(x, 0, y, where=(np.array(x) > two_right_tail) & (np.array(x) <= max(x)), facecolor='grey')
    #Mean +/- 1 sigma
    plt.axvline(y_chi2.mean(), c='red',label="mean")
    plt.axvline(y_chi2.mean()+y_chi2.std()*z, c='blue',label="+/-$\sigma_{gaus}$")
    plt.axvline(y_chi2.mean()-y_chi2.std()*z, c='blue')
    plt.xlabel("x")
    plt.ylabel("$\chi^{2}$")
    plt.legend(loc='upper right')
    plt.show()
plotItAll(x,y,y_chi2,z)

What we see now is that if we take deviations characteristic of two standard deviations or greater on the left and right of the distribution (these are the dark gray distributions) and we compare them to the expected variation that is up and down one standard deviation (the blue lines), we find that the p-values and the blue lines do not directly match to one another. This is because the standard deviation for an asymmetric distribution does not reflect the true width well, since the distribution is asymmetric. 

<a name='exercises_6_2'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_2) | [Next Section](#section_6_3) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.2.1</span>

In the last section, we were using the language of "p-values" and standard deviations. Now we will talk about things in terms of "confidence" and "z-score". 

What is the confidence level corresponding to a z-score of 4, for a normal distribution? Enter your answer as a number with precision 1e-6.

In [ ]:
#>>>EXERCISE: L6.2.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

pass


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.2.2</span>

Again, consider a normal distribution. What is the z-score for an event to be less than some value $x$, where we know the probability that the value is $>x$ is $p=0.40$?

Enter your answer as a number with precision 1e-3.

In [ ]:
#>>>EXERCISE: L6.2.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

pass


<a name='section_6_3'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.3 Another Example and Rules of Significance</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_2) | [Exercises](#exercises_6_3) | [Next Section](#section_6_4) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2024/block-v1:MITxT+8.S50.1x+3T2024+type@sequential+block@seq_LS6/block-v1:MITxT+8.S50.1x+3T2024+type@vertical+block@vert_LS6_vid3" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Slides</h3>

Run the code below to view the slides for this section, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L06/slides_L06_03.html" target="_blank">HERE</a>.

In [ ]:
#>>>RUN: L6.3-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L06/slides_L06_03.html', width=970, height=550)

<h3>Overview</h3>

Finally, let's consider a more complicated scenario where the shape of the distribution is very far away from a normal distribution. In this scenario, we want to show that conventional estimates for the width of a distribution break down dramatically. It's important to know this, since it can be very easy to use standard deviation to characterize the properties of a distribution. 

Let's consider a Cauchy distribution (described <a href="https://en.wikipedia.org/wiki/Cauchy_distribution" target="_blank">here</a>) and given by the form:

$$
\begin{equation}
f(x;x_{0},\gamma) = \frac{1}{\pi \gamma} \left(\frac{1}{\left(x-x_0\right)^{2} + \gamma^{2}}\right)
\end{equation}
$$

This function is available also in scipy stats with ```scipy.stats.cauchy```. For this function we will compare the Gaussian 1 standard deviation with the true Cauchy p-values to see what the differences are. 

In [ ]:
#>>>RUN: L6.3-runcell01

#NOTE: below we define a random seed for identical results between runs.
#These results may appear slightly different from the related video.


#Generate Cauchy data
np.random.seed(6)
y_cauchy = stats.cauchy.rvs(size=10000)


#choose z-score
z=1.0
two_right_tail = stats.cauchy.ppf(1 - stats.norm.cdf(-z))
two_left_tail = stats.cauchy.ppf(1 - stats.norm.cdf(z))
print('left/right-handed values corresponding to z-score:', two_left_tail,two_right_tail)


#print the mean and stdev of the distribution
fig, ax = plt.subplots(figsize=(12, 6))
y_cauchy = stats.cauchy.rvs(size=10000)
print('[mean of cauchy data, stdev of cauchy data]:', y_cauchy.mean(),z*y_cauchy.std())


#plot distribution
ymin = -300
ymax = 300
plt.xlim([ymin,ymax])
x = np.linspace(ymin, ymax, 10000)
y = stats.cauchy.pdf(x, loc=0, scale=1)
plt.plot(x, y, '--', c='black',label="f")
plt.hist(y_cauchy, histtype='stepfilled', edgecolor='k', alpha=0.4, color='gray', density=True,bins=10000)


#plot true-sigma
#plt.fill_between(x, 0, y, where=(np.array(x) > min(x)) & (np.array(x) <= two_left_tail), facecolor='grey',label='+/- true $\sigma$')
#plt.fill_between(x, 0, y, where=(np.array(x) > two_right_tail) & (np.array(x) <= max(x)), facecolor='grey')
plt.fill_between(x, 0, y, where=(np.array(x) < two_right_tail) & (np.array(x) >= two_left_tail), facecolor='grey',label='+/- gaus $\sigma$')


plt.axvline(y_cauchy.mean(), c='red',label="mean")
plt.axvline(y_cauchy.mean()+y_cauchy.std()*z, c='blue',label='+/- true $\sigma$')
plt.axvline(y_cauchy.mean()-y_cauchy.std()*z, c='blue')
plt.xlabel("x")
plt.ylabel("f(x)")
plt.legend(loc='upper right')
plt.yscale('log')
plt.show()

As you can see, the standard deviation(blue) and the p-values(dark gray) are completely different in scale. This is because the Cauchy distribution can have a few events that fluctuate very far in the tails away from the mean of the Cauchy distribution. 

In the preceding example with the Cauchy distribution, we found the positions of the left tail, `two_left_tail`, and the right tail, `two_right_tail`, such that the probability that a measurement drawn from the Cauchy distribution falls within the range `[two_left_tail,two_right_tail]` is equal to 0.6827. We chose the bounds to coincide with the confidence level of a Gaussian with z-score equal to 1.

To see that, run the code below.

In [ ]:
#>>>RUN: L6.3-runcell02

#calculate p_value from data
def get_p_value_data(idist, two_left_tail, two_right_tail):
  within_range = []
  for ielem in idist:
    if ielem <= two_right_tail and ielem >= two_left_tail:
      within_range.append(ielem)
  return len(within_range)/len(idist)


#Generate Cauchy data
np.random.seed(6)
y_cauchy = stats.cauchy.rvs(size=10000)

#choose z-score
z=1.0
two_right_tail = stats.cauchy.ppf(1 - stats.norm.cdf(-z))
two_left_tail = stats.cauchy.ppf(1 - stats.norm.cdf(z))

cauchy_p_value = get_p_value_data(y_cauchy, two_left_tail, two_right_tail)
print("p-value from left/right values and data:", cauchy_p_value)
print()

<a name='exercises_6_3'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_3) | [Next Section](#section_6_4) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.3.1</span>

What is the probability than an event drawn from a Cauchy distribution falls within 1-sigma of the mean? Use the data that we generated above to calculate your probability, and consider using the function `get_p_value_data()` (or write your own).

Enter your answer as a number with precision 1e-3.

In [ ]:
#>>>EXERCISE: L6.3.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#calculate p_value from data
def get_p_value_data(idist, two_left_tail, two_right_tail):
  within_range = []
  for ielem in idist:
    if ielem <= two_right_tail and ielem >= two_left_tail:
      within_range.append(ielem)
  return len(within_range)/len(idist)

#Generate Cauchy data
np.random.seed(6)
y_cauchy = stats.cauchy.rvs(size=10000)

#choose z-score
z=1.0
two_right_tail = #YOUR CODE HERE
two_left_tail = #YOUR CODE HERE

cauchy_p_value = get_p_value_data(y_cauchy, two_left_tail, two_right_tail)
print("p-value from left/right values and data:", cauchy_p_value)
print()


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.3.2</span>

What is the p-value that corresponds to a z-score of 1 for a Cauchy distribution? Enter your answer as a number with precision 1e-3.

In [ ]:
#>>>EXERCISE: L6.3.2
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

pass


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.3.3</span>

Which of the following options best describes a 3$\sigma$ detection:

- background noise
- evidence
- discovery


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.3.4</span>

Which of the following options best describes a 5$\sigma$ detection:

- background noise
- evidence
- discovery


<a name='section_6_4'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.4 Moments of Distributions and Mapping</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_3) | [Exercises](#exercises_6_4) | [Next Section](#section_6_5) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2024/block-v1:MITxT+8.S50.1x+3T2024+type@sequential+block@seq_LS6/block-v1:MITxT+8.S50.1x+3T2024+type@vertical+block@vert_LS6_vid4" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Slides</h3>

Run the code below to view the slides for this section, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L06/slides_L06_04.html" target="_blank">HERE</a>.

In [ ]:
#>>>RUN: L6.4-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L06/slides_L06_04.html', width=970, height=550)

<h3>Moments of distributions</h3>

So, you have seen that with certain types of distributions, things can deviate wildly from what is expected. Namely, in the case of a Cauchy or asymmetric distribution, the standard deviation of a data sample is not a reflection of the p-value of the distribution at all. To describe these sorts of distributions, there are a number of ways to mitigate this problem. The first is to introduce higher order moments of a pdf. 

Let's assume our distributions are centered about zero (we can always recenter them using a new variable $x'=x-\bar{x}$) ). Then, we can write moments of order $n$ as


$$
\begin{equation}
 \mu_{n}=m^{n}(x)=E[x^{n}p(x)] = \int_{-\infty}^{\infty} x^{n} p(x) dx
\end{equation}
$$


The mean is the moment of order 1, $E[p(x)]=m^{1}(x)$, and the variance of the distribution is the 2$^{\textrm{nd}}$ order moment, $V[p(x)]=m^{2}(x)$. These are the two properties of a distribution that you already know and love, but we can keep going to higher order moments in order to describe more complicated distributions. The next moment, called "skewness", tells you how asymmetric a distribution is; $\mathrm{Skew}=m^{3}(x)$, and going further we have the $\mathrm{Kurtosis}=m^{4}(x)$, which tells you how important the tails of a distribution are. 

Lots of different distributions can be found in data. Here is a <a href="http://bois.caltech.edu/dist_stories/t3b_probability_stories.html" target="_blank">"story book of distributions."</a>


Note that this link is also a nice concise summary of a wide range of distributions, including their functional forms and a brief description of their properties.

Let's take a look at the moments of our two example distributions discussed above. 


In [ ]:
#>>>RUN: L6.4-runcell01

def raw_moment(X, k, c=0):
    return ((X - c)**k).mean()

def central_moment(X, k):
    if k == 1:
        return X.mean()
    return raw_moment(X=X, k=k, c=X.mean())

def print_moments(X,label):
    print(label+" mean:",central_moment(X,1))
    print(label+" var:" ,central_moment(X,2))
    print(label+" skew:",central_moment(X,3))
    print(label+" kurtosis:",central_moment(X,4))

N=1000000
y_norm = stats.norm.rvs(size=N)
print_moments(y_norm,"normal")

df=9
y_chi2 = stats.chi2.rvs(size=N,df=df)
print_moments(y_chi2,"chi2 df "+str(df))

y_cauchy = stats.cauchy.rvs(size=N)
print_moments(y_cauchy,"cauchy")


Now, the nice thing about moments is that you can use these values to map any distribution to any other distribution. 

I am not really doing justice to the above, but moments are in fact incredibly useful. A chunk of my PhD thesis is dedicated to using moments of a Gaussian (cumulants) to model distributions with a small amount of data. 

In light of this, what we can imagine doing is to use a transform, like we do with a Fourier transform. Through the use of z-scores and moments, you can transform one probability distribution function into any another. The strategy here is to match quantiles, or z-scores, with each other, such that you have a transform from $x\rightarrow x^{\prime}$, which does this matching for distributions $p_{1}$ and $p_2$:

$$
\begin{equation}
\int_{-\infty}^{x} p_1(x) dx = \int_{-\infty}^{x^\prime} p_2(x^{\prime}) dx^{\prime}
\end{equation}
$$

To do this mapping, there are many different approaches. All of these approaches do roughly the same thing, map one pdf to another pdf.  One such approach of mapping distributions is the <a href="https://en.wikipedia.org/wiki/Power_transform" target="_blank">box-cox method</a>, which aims to use the moments to transform any distribution into a Gaussian distribution. Let's see how well it works.

To do this we are going to sample a lognormal distribution using `stats.lognorm`.  This distribution is very much not a Gaussian, as we will see. 

In [ ]:
#>>>RUN: L6.4-runcell02

x = stats.lognorm.rvs(s=1, loc=0, scale=5, size=1000, random_state=4)

# plot
def plotdistandq(x,xaxis,evals):
    fig, axes = plt.subplots(1, 2, figsize=(10, 3))
    axes[0].hist(x,density=True)
    axes[0].plot(xaxis,evals)
    stats.probplot(x, dist=stats.norm, plot=axes[1])
    fig.tight_layout()

xaxis=np.linspace(0,80, 1000)
evals=stats.lognorm.pdf(xaxis,s=1, loc=0, scale=5)
plotdistandq(x,xaxis,evals)
# box-cox transform
xt, lmbda = stats.boxcox(x)

#now plot Gaussian
gxaxis = np.linspace(-2, 5, 1000)
gevals = stats.norm.pdf(gxaxis,xt.mean(),1) 
plotdistandq(xt,gxaxis,gevals)

stats.probplot(xt, dist=stats.norm, plot=axes[1])
fig.tight_layout()

These types of transforms are particularly useful when we are trying to match a simulated distribution with a data distribution. If both the simulated and true distributions are trying to describe the same thing, you can use a transform like this to map one distribution onto the other.

<h3>Applying this Method to Cauchy Distribution</h3>

Let's now do this for a more complicated Cauchy distribution, and see how it behaves. In fact, let's take the absolute value of it. 

In [ ]:
#>>>RUN: L6.4-runcell03

x = np.abs(stats.cauchy.rvs(size=10000))

xaxis=np.linspace(0,80, 1000)
evals=np.abs(stats.cauchy.pdf(xaxis))
plotdistandq(x,xaxis,evals)
# box-cox transform
xt, lmbda = stats.boxcox(x)

#now plot Gaussian
gxaxis = np.linspace(-2, 5, 1000)
gevals = stats.norm.pdf(gxaxis,xt.mean(),1) 
plotdistandq(xt,gxaxis,gevals)

stats.probplot(xt, dist=stats.norm, plot=axes[1])
fig.tight_layout()

print("mean:",xt.mean())

The Cauchy distribution is a hard distribution to model because of its giant tails. However, in the bulk it is very much Gaussian, so using a box-cox does surprisingly well in the cental region. However, in the far tails it starts to deviate quite a bit.

<a name='exercises_6_4'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_4) | [Next Section](#section_6_5) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.4.1</span>

Take a $\chi^2$ distribution with 1 degree of freedom, which is very asymmetric, and map it onto a Gaussian. What is the mean of this distribution?

Complete the code below, which uses a random seed to ensure your answer will match ours. Enter your answer as a number with precision 1e-3.


In [ ]:
#>>>EXERCISE: L6.4.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

np.random.seed(10)
x = stats.chi2.rvs(size=10000,df=1)

#YOUR CODE HERE

<a name='section_6_5'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.5 Monte Carlo Integration</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_4) | [Exercises](#exercises_6_5) | [Next Section](#section_6_6) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2024/block-v1:MITxT+8.S50.1x+3T2024+type@sequential+block@seq_LS6/block-v1:MITxT+8.S50.1x+3T2024+type@vertical+block@vert_LS6_vid5" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Slides</h3>

Run the code below to view the slides for this section, which are discussed in the related video. You can also open the slides in a separate window <a href="https://mitx-8s50.github.io/slides/L06/slides_L06_05.html" target="_blank">HERE</a>.

In [ ]:
#>>>RUN: L6.5-slides

from IPython.display import IFrame
IFrame(src='https://mitx-8s50.github.io/slides/L06/slides_L06_05.html', width=970, height=550)

<h3>Numerical Integration</h3>

You have probably noticed that I didn't bother to compute integrals to get the moments above, I just computed the means of the moments of sampled distributions. Instead, I did what is referred to as Monte-Carlo Integration (or more generally bootstrapping) where I just sampled a distribution and integrated by sampling. Namely, 

$$
\begin{equation}
E[x^{n}p(x)] = \int_{-\infty}^{\infty} x^{n} p(x) dx = \frac{1}{N}\sum_{i=1}^{N} x_{i}^{n}
\end{equation}
$$

where here the $x_{i}\in p(x)$ are sampled from the probability distribution function. So what happens, when we have a distribution, but we don't know the analytic form. How can we sample it?  

There are a lot of ways to do this, perhaps the best well known is Markov Chain Monte Carlo (MCMC). However the simplest is to just turn our distribution into a 2D image and randomly sample points on the image. Instead of writing the points out, let's just do it. 

To see how this works, let's walk through a calculation of integrating a quarter circle. We know that the integral is given by $A=\frac{\pi}{4}$, so we can check our math. What we will do is 

1. Sample randomly in x
2. Sample randomly in y
3. Check to see if x and y are within our quarter circle. 
4. Compute the number of samples within our quarter circle, compared to all points. 

Let's look at the code. Note that since we are randomly sampling, our uncertainty on our random sample will just be the poisson uncertainty or $\frac{a}{\sqrt{N_{\rm samples}}}$.


In [ ]:
#>>>RUN: L6.5-runcell01

import math

#First let's just compute the area of a quarter circle with radius 1
def quarterarea(iN):
    area=0
    lXin = np.array([])
    lYin = np.array([])
    lXout = np.array([])
    lYout = np.array([])
    for i0 in range(iN):
        #Sample X and Y
        pX = np.random.uniform(0,1)
        pY = np.random.uniform(0,1)
        #Check if its radius is in 1
        if math.sqrt(pX**2+pY**2) < 1:
            lXin = np.append(lXin,pX)
            lYin = np.append(lYin,pY)
            area += 1 # count it
        else:
            lXout = np.append(lXout,pX)
            lYout = np.append(lYout,pY)
    return (float(area)/float(iN)),lXin,lYin,lXout,lYout

#sample points
lN=1000
#lN=100000
a,lXin,lYin,lXout,lYout=quarterarea(lN)
print("Pi (4*area):",a*4,"+/-",4*a/math.sqrt(lN)) #gotta put an uncertainty
plt.plot(lXin,lYin,marker='.',linestyle = 'None')
plt.plot(lXout,lYout,marker='.',linestyle = 'None')
plt.show()

The idea with Monte Carlo integration is that we calculate an integral by evaluating the function. We don't actually have to compute the integral. This avoids what is potentially a very complicated step.  As you probably well know, computing integrals can be very hard, this way gets at our answer just through functional evaluation. 

Now, let's compute the integral of some arbitrary function $y=f(x)$. As we did with our circle, we can compute the integral by plotting this function over a range. Here we will do $-6 < x < 3$. From this, what we will do is sample over a 2D Gaussian distribution of width given by the x and y ranges and center given by the minimum x and y values. Why this distribution to sample? Because we can.

In [ ]:
#>>>RUN: L6.5-runcell02

from scipy import optimize as opt 

#use this random seed
np.random.seed(10)

#Now let's consider integrating some random function
def f(x):
    return x**4 + 3*(x-2)**3 - 15*(x)**2 + 1

#Now let's multiply it by -1 to make the range calculation fast
def fneg(x):
    return -1*(x**4 + 3*(x-2)**3 - 15*(x)**2 + 1)

#First thing is to define a range in x
xmin=-6
xmax=3
x = np.linspace(xmin, xmax, 100)
#plt.plot(x, f(x));

#Now we need to find a range in y
sol=opt.minimize_scalar(f,bounds=(xmin, xmax), method='Brent')
ymin=sol.fun
#y-max is to get the minimum of negative f
sol=opt.minimize_scalar(fneg,bounds=(xmin, xmax), method='Brent')
ymax=-1*sol.fun
print('[ymin,ymax]:', ymin, ymax)

lN=100000
#now, let's sample a 2D grid y-min and y-max and compute the integral
lXin = np.array([])
lYin = np.array([])
lXout = np.array([])
lYout = np.array([])
for i0 in range(lN):
    #Try a uniform distribution
    pX = abs(xmax-xmin)*np.random.uniform(0,1)+xmin
    pY = abs(ymax-ymin)*np.random.uniform(0,1)+ymin
    #Try a normal distribution
    #pX = abs(xmax-xmin)*np.random.normal(0,1)+xmin
    #pY = abs(ymax-ymin)*np.random.normal(0,1)+ymin
    pYMin = f(pX)
    if pY < pYMin:
        lXin = np.append(lXin,pX)
        lYin = np.append(lYin,pY)
    else:
        lXout = np.append(lXout,pX)
        lYout = np.append(lYout,pY)


plt.plot(lXin,lYin,marker='.',linestyle = 'None', color='orange')
plt.plot(lXout,lYout,marker='.',linestyle = 'None', color='green')
plt.axvline(sol.x, c='red', lw=3)
plt.plot(x, f(x), 'b-', lw=3)
#plt.ylim(-800,0)
#plt.xlim(-6,3)
plt.show();

From the above scenario, we can see that the orange points are below the line from roughly -800 to 0, and the green points are above the line. We can make a histogram of this.

In [ ]:
#>>>RUN: L6.5-runcell03

_,bins,_=plt.hist(lXin,bins=10,alpha=0.5,density=True, label='below')
plt.hist(lXout,alpha=0.5,bins=bins,density=True, label='above')
#plt.xlim(-6,3)
plt.legend(loc=1)
plt.show();

print("number below:",len(lXin),"number above:",len(lXout))
#Note, these numbers will be slightly different for each run
#due to random generation of data

We have to be careful with the above integral if we are sampling a Gaussian distribution. In that case, our sampling is biased, not uniform, which means our integral will not be correct, and it will be biased towards the Gaussian we chose. We can write this out as the product of our function times the phase space we are sampling. In this case, the phase space is given by the Hessian of a 2D Gaussian $\frac{\partial^{2} G(x,y) }{\partial x \partial y} $

$\mathcal{I}=\int_{\rm space} f(x,y) \frac{\partial^{2} G(x,y) }{\partial x \partial y} dx dy$

This procedure is known as "Area-based" sampling, and is considered a method of Monte-Carlo Integration. Monte-Carlo integration is a rich field. All high energy physics simulations are based on it. Basically, the function we sample from starts with a collision and computing the probability that this could be any other collision. We then proceed to put this single collision through a point by point simulation, of each particle going through all the detectors. Finally we aggregate our distributions based on this. You will see the usefulness of Monte Carlo Simulation later on. 

<a name='exercises_6_5'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_5) | [Next Section](#section_6_6) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.5.1</span>

Compute the fraction of events above the line corresponding to the function that we defined previously:

<pre>
def f(x):
    return x**4 + 3*(x-2)**3 - 15*(x)**2 + 1
</pre>

Specifically, sample `100000` points from a uniform distribution over the x and y ranges specified above (i.e., $-6 < x < 3$ and $f(-6) < y < f(3)$ to compute this fraction.

Enter your answer as a number with precision 1e-3.

Hint: Use the `lXout` value from `L6.5-runcell02`. This means the solution is only 3 lines.

In [ ]:
#>>>EXERCISE: L6.5.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#YOUR CODE HERE
#Note, if you are computing the distributions again, use the following random seed
np.random.seed(10)
pass


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.5.2</span>

How would you compute the integral of this function over the range `[-6,3]`? Select the correct answer below:

- The integral is equal to the number of points above the line defined by our function.
- The integral is equal to the number of points between zero and the line defined by our function.
- The integral is equal to the fraction of points that we calculated, times the area of the region that we defined.
- The integral is equal to the fraction of points between zero and our function, times the area of the region where the simulation is carried out, which must use `0` as an upper bound.


>#### Follow-up 6.5.2a (ungraded)
>
>Calculate the integral numerically. Remember to correctly apply a negative sign, if applicable.
>
>How does your numerically calculated value compare to the expected value? You can explicitly solve the integral mathematically, or use a built-in `scipy` integration method to compare to your Monte Carlo integration.

In [ ]:
#>>>FOLLOW-UP: L6.5.2a
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

#YOUR CODE HERE
pass


<a name='section_6_6'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.6 Returning to Fitting Supernova Data</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_5) | [Exercises](#exercises_6_6) | [Next Section](#section_6_7) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2024/block-v1:MITxT+8.S50.1x+3T2024+type@sequential+block@seq_LS6/block-v1:MITxT+8.S50.1x+3T2024+type@vertical+block@vert_LS6_vid6" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>A more sophisticated fit</h3>

Now that we have gone on an excursion to understand properties of fits, let's go ahead and analyze our supernovae data, and try to pull in all of the information that we can. Let's first look at our linear fit.  One sec, while we load it all:


In [ ]:
#>>>RUN: L6.6-runcell01
import math
import numpy as np
import csv
import matplotlib.pyplot as plt
from scipy import stats

#Let's try to understand how good the fits we made in last Lesson are, let's load the supernova data again
label='data/L04/sn_z_mu_dmu_plow_union2.1.txt'

def distanceconv(iMu):
    power=iMu/5+1
    return 10**power

def distanceconverr(iMu,iMuErr):
    power=iMu/5+1
    const=math.log(10)/5.
    return const*(10**power)*iMuErr

#only reads data up to z=0.1.
def load(iLabel,iMaxZ=0.1):
    redshift=np.array([])
    distance=np.array([])
    distance_err=np.array([])
    with open(iLabel,'r') as csvfile:
        plots = csv.reader(csvfile, delimiter='\t')
        for row in plots:
            if float(row[1]) > iMaxZ:
                continue
            redshift = np.append(redshift,float(row[1]))
            distance = np.append(distance,distanceconv(float(row[2])))
            distance_err = np.append(distance_err,distanceconverr(float(row[2]),float(row[3])))
    return redshift,distance,distance_err  
        
#Now let's run the regression again
def variance(isamples):
    mean=isamples.mean()
    n=len(isamples)
    tot=0
    for pVal in isamples:
        tot+=(pVal-mean)**2
    return tot/n

def covariance(ixs,iys):
    meanx=ixs.mean()
    meany=iys.mean()
    n=len(ixs)
    tot=0
    for i0 in range(len(ixs)):
        tot+=(ixs[i0]-meanx)*(iys[i0]-meany)
    return tot/n

def linear(ix,ia,ib):
    return ia*ix+ib

redshift,distance,distance_err=load(label)
var=variance(redshift)
cov=covariance(redshift,distance)
A=cov/var
const=distance.mean()-A*redshift.mean()
xvals = np.linspace(0,0.1,100)
yvals = []
for pX in xvals:
    yvals.append(linear(pX,A,const))

plt.plot(xvals,yvals)
plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.xlabel("z(redshift)")
plt.ylabel("distance(pc)")
plt.show()

Now that we have loaded the data, let's actually look at the residuals. What this means is that we are going to compute the difference between the mean predicted value in $y$ and the true data points. 

In [ ]:
#>>>RUN: L6.6-runcell02

def residualsComp(redshift,distance,distance_err):
    #Compute residuals
    residuals=np.array([])
    for i0 in range(len(redshift)):
        pResid=linear(redshift[i0],A,const)-distance[i0]
        residuals = np.append(residuals,pResid/distance_err[i0])
    
    #Make a histogram
    y0, bin_edges = np.histogram(residuals, bins=30)
    bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
    norm0=len(residuals)*(bin_edges[-1]-bin_edges[0])/30.
    plt.errorbar(bin_centers,y0/norm0,yerr=y0**0.5/norm0,marker='.',drawstyle = 'steps-mid')
    
    #Plot a Gaussian
    k=np.arange(bin_edges[0],bin_edges[-1],0.05)
    normal=stats.norm.pdf(k,0,1)
    #First let's look at the moments 
    normalpoints=stats.norm.rvs(0,1,1000)
    print_moments(residuals,"residuals")
    print_moments(normalpoints,"normal distribution")

    #Now let's plot it
    plt.plot(k,normal,'o-')
    plt.xlabel("number of successes")
    plt.ylabel("probability")
    plt.show()
    return residuals

residuals=residualsComp(redshift,distance,distance_err)

What we see is that the mean of the residuals is very close to 0 with a variance of 1, and a skew and kurtosis really close to a normal distribution. This looks likes our residuals are Gaussian (in fact, we have plotted a Gaussian distribution on top of our fit). What do you think that means for our fit? 

Recall that a $\chi^{2}$ distribution is defined as a sum of $N$ Gaussian distributed variables with mean $0$ and width $1$. For a fit this means that if we compute the residuals divided by their uncertainties, we get the value

$\chi^{2} = \sum_{i} \left(\frac{y_{i}-f(x{i})}{\sigma}\right)^{2}$ 

which should be distributed by a $\chi^{2}$ distribution. This is why we call this parameter $\chi^2$. I know this sounds a little circular, but it does make sense.  Anyway the fit should be distributed by a $\chi^{2}$ distribution given by the number of degrees of freedom, which in this case is the number of points minus the number of parameters we floated in the fit, which in this case is $2$. 

Let's check it out. 

In [ ]:
#>>>RUN: L6.6-runcell03

#now let's look at the chi2
chi2=np.sum(residuals**2)

print("Total chi2:",chi2,"NDOF",len(residuals)-2)
print("Normalized chi2:",chi2/(len(residuals)-2))
print("Probability of chi2:",1-stats.chi2.cdf(chi2,(len(residuals)-2)))
print()

#Let's plot it for good measure too
x = np.linspace(0,len(residuals)*2)
chi2d=stats.chi2.pdf(x,len(residuals-2)) # 40 bins
plt.plot(x,chi2d,label='chi2')
plt.axvline(chi2, c='red')
plt.legend(loc='lower right')

ndof=(len(residuals)-2)
chi2ppf0=stats.chi2.ppf(0.5,ndof)
chi2ppf1=stats.chi2.ppf(0.15,ndof)
chi2ppf2=stats.chi2.ppf(0.85,ndof)
#chi2ppf1=stats.chi2.ppf(0.025,ndof)
#chi2ppf2=stats.chi2.ppf(1-0.025,ndof)
print("Central Vvalue",chi2ppf0)
print("Sigma Low",chi2ppf1-chi2ppf0)
print("Sigma High",chi2ppf2-chi2ppf0)

plt.axvline(chi2ppf1, c='blue')
plt.axvline(chi2ppf2, c='blue')
plt.show()

We see that the $\chi^{2}$ value is very close to the number of degrees of freedom, with a normalized $\chi^{2}/NDF \approx1$, that is a legitimately good fit! Let's loosen the data by increasing the overall range of redshifts that we will fit, since here we only loaded and fit redshifts up to 0.1.

<a name='exercises_6_6'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_6) | [Next Section](#section_6_7) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.6.1</span>

Fill in the blank: Anything above ___ for chi squared probability is a sign of a good fit. Enter your answer as a number with precision 1e-2.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.6.2</span>

Fill in the blank: For a normal distribution, a good fit has a normalized chi squared value near ___ . Enter your answer as a number with precision 1e-2.

### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.6.3</span>

Ok, now let's repeat the linear fit above without a cut on the redshift. Let's run our full slew of metrics on it like we did before. Complete the code below and analyze the output.

What is the value of the normalized (or reduced) chi-square metric? Is this a good fit or a bad fit? Enter your answer as a number with precision 1e-2.

In [ ]:
#>>>RUN: L6.6.3

#run regression
redshift,distance,distance_err=load(label,10000)
var=variance(redshift)
cov=covariance(redshift,distance)
A=cov/var
const=distance.mean()-A*redshift.mean()
xvals = np.linspace(0,1.4,100)
yvals = []
for pX in xvals:
    yvals.append(linear(pX,A,const))

#plot it
plt.plot(xvals,yvals)
plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.xlabel("z(redshift)")
plt.ylabel("distance(pc)")
plt.show()

residuals=residualsComp(redshift,distance,distance_err)
#now let's look at the chi2
#YOUR CODE HERE

<a name='section_6_7'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.7 Fitting with a More Accurate Model</h2>  

| [Top](#section_6_0) | [Previous Section](#section_6_6) | [Exercises](#exercises_6_7) | [Next Section](#section_6_8) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2024/block-v1:MITxT+8.S50.1x+3T2024+type@sequential+block@seq_LS6/block-v1:MITxT+8.S50.1x+3T2024+type@vertical+block@vert_LS6_vid7" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

<h3>Overview</h3>

When our fits don't work well, what we need to do is come up with a better fit function. There are many ways to come up with a better function. Sometimes we just guess what a better function is. However, the best way to come up with a better function is to use our knowledge of the data and physics. 

When we expand the range of redshifts, what we are doing is looking at a larger range of the universe. That means that we need to come up with a better model of the universe. Fortunately, in this case there has been a lot of work towards building a better model of the universe. This work has led us to the so called Friedmann equations for the expansion of the universe. 

We can write the <a href="https://en.wikipedia.org/wiki/Friedmann_equations" target="_blank">Friedmann equations</a> in terms of the Hubble constant $h$ relative to its current value $h_{0}$ and the density parameters of the universe for matter $\Omega_{m}$, Dark Matter $\Omega_{DM}$, Radiation density $\Omega_{r}$, curvature of the universe $\Omega_{\kappa}$ and dark Energy $\Omega_{\Lambda}$ as  

$\left(\frac{\dot{h}}{h_0}\right)^{2} = (\Omega_{m} + \Omega_{\rm DM})a^{-3}+\Omega_{r}a^{-4} + \Omega_{\kappa} a^{-2} + \Omega_{\Lambda}$

Here, the parameter $a$ is the scale parameter of the universe. This is often referred to as the cosmic scale. The way to think about this is that this is the average distance between all the galaxies in the universe at any period of time in the uiverse. When the big bang happened the scale $a\rightarrow0$ or in otherwords all the galaxies are next to each other. As the universe expands, $a$ gets larger and at some point $a\rightarrow\infty$ or if the universe collapses back in on itself in a phenomenon known as the big crunch $a$ would actually tend towards 0 again. To make life simple, we tend to take $a=1$ for our current period of time. What this means is we are choosing our units conveniently so that this distance is 1 now in our galactic equation. At our time we also have that Hubble's constant $h$, which varies with the age of the universe thus becomes $h_{0}$. As a result, we can write the above equation as 

$$1 = (\Omega_{m} + \Omega_{\rm DM})+\Omega_{r}+ \Omega_{\kappa} + \Omega_{\Lambda}$$

Now, we can further simplify this by our knowledge of the universe $\Omega_{r}$ is the density from radiation. We don't see a huge amount of radiation appearing in our current universe, so we can set $\Omega_{r}=0$. In reality, we can actually test this. To make life simpler, we can't really separate $\Omega_{m}$ from $\Omega_{\rm DM}$, so let's call this $\Omega_{M}$. Finally, for now let's set $\Omega_{\kappa}=0$. You will investigate this later. 

This gives us the following equation: 

$$1 = \Omega_{M}+ \Omega_{\Lambda} {\rm  ~or~ } \Omega_{\Lambda} = 1- \Omega_{M}$$  

and furthermore, we can rewrite the above equation as 

$$\left(\frac{\dot{h}}{h_0}\right)^{2} = (\Omega_{M})a^{-3} + 1-\Omega_{M}$$

So at this point you might be asking what does this have to do with our data, which is distance versus redhift. It turns out we can define both distance and redshift by parameters in the above equation. First, let's define distance. What we are looking for is the distance that light travels to a galaxy far away. As we go further in distance, we go earlier in the universe. So if we know the age of the universe $t$ relative to our current time, we know that the light travelled $d=ct$. Anyway, let's derive $t$ as a function of our scale parameter $a$ to see how it evolves. If it's not exactly clear what is going on, just wait. 

Hubbles constant $h$ is a measure of the rate of expansion of the universe. This is just equal to the time derivative of the scale parameter $a$, or in other words:

$$\frac{da}{dt}= h =h_{0} \sqrt{(\Omega_{M})a^{-3} + 1-\Omega_{M}}$$

From this we can separate our equation into parts to notice that 

$$\int h_{0} dt = h_{0} t = \int \frac{da}{\sqrt{(\Omega_{M})a^{-3} + 1-\Omega_{M}}}$$

Finally, the distance that light travels for a time $t$ is $d=ct$, so we can define distance as 

$$ d(a) = ct = \frac{c}{h_{0}}\int \frac{da}{\sqrt{(\Omega_{M})a^{-3} + 1-\Omega_{M}}}$$

This $d$ is often referred to as the comoving distance, this is the distance on your y axis. 

Now the x-axis is redshift. Redshift is how much light got stretched compared to the present day. As the universe expands, it also stretches light outwards. That's because light moving in an expanding medium will stretch out. The redshift is thus proportional to our scale parameter $a$. Since the redshift is currently $0$ and proportional to $a$ we have that for $a=1$ $z=0$, and for $a\rightarrow0$ $z\rightarrow\infty$ or $z\propto 1/a$, this allows us to immediatly write the value for $z$ as below

$$z = \frac{1}{a}-1$$

Likewise

$$a=\frac{1}{1+z}$$

this finally gives us a function $d(z)$

$$ d(z) = ct = \frac{c}{h_{0}}\int_{0}^{z} \frac{dz^{\prime}}{\sqrt{\Omega_{M}(1+z^{\prime})^{3} + 1-\Omega_{M}}}$$

Where now we have finally put bounds on the integral to correspond to the actual value of $z$ we are choosing. Note that we integrate over a range of $z^{\prime}$ to get the value of $z$ that we want to use. The above is referred to as the co-moving distance. 

Finally, before we actually go to our data, we need to correct the co-moving distance by a correction to account for the fact that we are measuring the luminosity distance and not the co-moving distance.  The luminosity distance is defined as  the change in brightness of a star, or the flux $S$ at any given point is defined as the surface area of the luminosity distance times the luminosity $L$

$$ 4\pi d_{L}^{2} L = S$$  

Now since the universe is expanding as light is moving out, the light is getting redishifted or in otherwords since the universe is expanding it takes longer for the same wave to pass by you since light moves at a constant speed, but space is expanding. This additional length is just the additional redshift scale scale ie $d_{L} = (1+z) d_{\rm original}$. This means we finally write our equation as: 

$$ d(z) = ct^{\prime} = (1+z)ct = (1+z)\frac{c}{h_{0}}\int_{0}^{z} \frac{dz^{\prime}}{\sqrt{\Omega_{M}(1+z^{\prime})^{3} + 1-\Omega_{M}}}$$


<h3>Numerical properties of the universe</h3>

We can take the above integral and now write the whole thing numerically. This is just going to be the speed of light over Hubble's constant in the right units $(c/h_{0})$ multiplied by the integral and the multiplied by $1+z$. That's it. 

To perform the integral, all we do is divide our range into 100 pieces and no a numerical integral by doing a for loop. In other words, we write 

$$ d(z) = \frac{c}{h_{0}}(1+z) \sum_{i=0}^{i=100} \frac{dz}{\sqrt{\Omega_{M}(1+z_{i}^{\prime})^{3} + 1-\Omega_{M}}}$$

where $dz=z/100$ and $z_{i}= dz \times i$. Let's go ahead and write this out. 



In [ ]:
#>>>RUN: L6.7-runcell02

#We are not going to plot the fit first, let's just use our barrage of statistics to check if its ok
def hubble(z,Om):
    pVal=Om*(1+z)**3+(1.-Om)
    return np.sqrt(pVal)

def lumidistance(z,h0,Om):
    integral=0
    nint=100
    for i0 in range(nint):
        zp=z*float(i0)/100.
        dz=z/float(nint)
        pVal=1./(1e-5+hubble(zp,Om))
        integral += pVal*dz
    d=(1.+z)*integral*(1e6*3e5/h0)
    return d

print("test Lumi",lumidistance(1,70,0.3))


<a name='exercises_6_7'></a>     

| [Top](#section_6_0) | [Restart Section](#section_6_7) | [Next Section](#section_6_8) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.7.1</span>

How would more dark energy change the luminosity distance over redshift? You can approach this problem mathematically, or simply change the apppropriate parameter in the function `lumidistance()` to compute the luminosity distance at different values.

Based on your observations, if there is more dark energy, the luminosity distance should:

- get larger
- get smaller
- stay the same


In [ ]:
#>>>EXERCISE: L6.7.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

pass

<a name='section_6_8'></a>
<hr style="height: 1px;">

## <h2 style="border:1px; border-style:solid; padding: 0.25em; color: #FFFFFF; background-color: #90409C">L6.8 Fit to Full Cosmological Model</h2>     

| [Top](#section_6_0) | [Previous Section](#section_6_7) | [Exercises](#exercises_6_8) |


*The material in this section is discussed in the video **<a href="https://courses.mitxonline.mit.edu/learn/course/course-v1:MITxT+8.S50.1x+3T2024/block-v1:MITxT+8.S50.1x+3T2024+type@sequential+block@seq_LS6/block-v1:MITxT+8.S50.1x+3T2024+type@vertical+block@vert_LS6_vid8" target="_blank">HERE</a>.** You are encouraged to watch that video and use this notebook concurrently.*

Now let's go and run our fit function that included the parameters of the universe. As a first pass, let's just pipe this through lmfit and see if it works. 

In [ ]:
#>>>RUN: L6.8-runcell01

import lmfit

model  = lmfit.Model(lumidistance)
p = model.make_params(h0=70,Om=0.2)
result = model.fit(data=distance, params=p, z=redshift, weights=1./distance_err)
lmfit.report_fit(result)
plt.figure()
result.plot()

Now, since we are experts at fitting, let's go ahead and compute the log likelihood . We are going to minimize the (negative of) loglikelihood with `scipy.optimize` in order to do the fit.

In [ ]:
#>>>RUN: L6.8-runcell02
from scipy import optimize as opt 

def loglike(x):
    lTot=0
    for i0 in range(len(redshift)):
        xtest=lumidistance(redshift[i0],x[0],x[1])
        #lTot = lTot+(distance[i0]-xtest)**2
        lTot = lTot+((1./distance_err[i0])**2)*(distance[i0]-xtest)**2
    return lTot #*0.5 The above is 2 times loglike

def residuals(x):
    residuals=np.array([])
    for i0 in range(len(redshift)):
        pResid=lumidistance(redshift[i0],sol.x[0],sol.x[1])-distance[i0]
        residuals = np.append(residuals,pResid/distance_err[i0])
    return residuals


x0 = np.array([60.,0.2])
ps = [x0]
bnds = ((0, 1000), (0, 1.0))
sol=opt.minimize(loglike, x0,bounds=bnds, tol=1e-6)
print(sol)
residuals=residuals(sol.x)
print_moments(residuals,"residuals")
chi2=np.sum(residuals**2)
print("Total chi2:",chi2,"NDOF",len(residuals)-2)
print("Normalized chi2:",chi2/(len(residuals)-2))
print("Probability of chi2:",1-stats.chi2.cdf(chi2,(len(residuals)-2)))

#Let's plot it for good measure too
x = np.linspace(0,len(residuals)*2)
chi2d=stats.chi2.pdf(x,len(residuals-2)) # 40 bins
plt.plot(x,chi2d,label='chi2')
plt.axvline(chi2, c='red')
plt.legend(loc='lower right')
plt.show();

What can we say about this fit, is there something off?

Let's plot the residuals and the fit function and also scan the likelihood for our parameter uncertainties. There is one thing off, can you figure it out?


In [ ]:
#>>>RUN: L6.8-runcell03

#Plot it against the data
xvals = np.linspace(0,1.4,100)
yvals = []
for pX in xvals:
    yvals.append(lumidistance(pX,sol.x[0],sol.x[1]))

plt.errorbar(redshift,distance,yerr=distance_err,marker='.',linestyle = 'None', color = 'black')
plt.plot(xvals,yvals)
plt.show()

#Histogram the residuals
y0, bin_edges = np.histogram(residuals, bins=30)
bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
norm0=len(residuals)*(bin_edges[-1]-bin_edges[0])/30.
plt.errorbar(bin_centers,y0/norm0,yerr=y0**0.5/norm0,marker='.',drawstyle = 'steps-mid')
k=np.arange(bin_edges[0],bin_edges[-1],0.05)
normal=stats.norm.pdf(k,0,1)
plt.plot(k,normal,'o-')
plt.show()

x = np.linspace(len(residuals)*0.5,len(residuals)*1.5)
chi2d=stats.chi2.pdf(x,len(residuals-2)) # 40 bins
plt.plot(x,chi2d,label='chi2')
plt.axvline(chi2, c='red')
plt.legend(loc='lower right')
plt.show()




<a name='exercises_6_8'></a>   

| [Top](#section_6_0) | [Restart Section](#section_6_8) |


### <span style="border:3px; border-style:solid; padding: 0.15em; border-color: #90409C; color: #90409C;">Ex-6.8.1</span>

So, given the Friedmann equations, we can add back the curvature term:

$$ d(z) = ct^{\prime} = (1+z)ct = (1+z)\frac{c}{h_{0}}\int_{0}^{z} \frac{dz^{\prime}}{\sqrt{\Omega_{M}\left(1+z^{\prime}\right)^{3} + \Omega_{\kappa}\left(1+z^{\prime}\right)^{2}+ 1-\Omega_{M}-\Omega_{\kappa}}}$$

Adjust the `lumidistance()` function to fit for curvature. What is the value of $\Omega_{\kappa}$ if we perform a new fit to the data? What do you think this is implying? 

Enter your answer as a number with precision 1e-2.


In [ ]:
#>>>EXERCISE: L6.8.1
# Use this cell for drafting your solution (if desired),
# then enter your solution in the interactive problem online to be graded.

def hubble_curve(z,Om,OmK):
    return #YOUR CODE HERE

def lumidistance_curve(z,h0,Om,OmK):
    return #YOUR CODE HERE


model  = lmfit.Model(lumidistance_curve)
p = model.make_params(h0=70,Om=0.2,OmK=0.0)
result = model.fit(data=distance, params=p, z=redshift, weights=1./distance_err)
lmfit.report_fit(result)
result.plot();
